In [100]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob 
import os
import warnings
import networkx as nx
import matplotlib.colors as mcolors
warnings.simplefilter(action='ignore', category=FutureWarning)
from scipy.stats import pearsonr
from sklearn.covariance import GraphicalLassoCV
from networkx.generators.community import gaussian_random_partition_graph
import scipy.stats as stats
from scipy.stats import ttest_ind
import nibabel as nib 
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import tqdm

## This notebook is related to mri old project 
* Basically want to find the mean GMV for each region and see if they are different between groups and which regions have the biggest difference 

In [101]:
adni_progs_pruned = pd.read_csv('/data2/MRI_PET_DATA/graph/ADNI/mri_atlas/roi/ADNI_pruned_prog_csv.csv')
adni_stable_pruned = pd.read_csv('/data2/MRI_PET_DATA/graph/ADNI/mri_atlas/roi/ADNI_pruned_stable_csv.csv')
print(adni_stable_pruned[0:3])

   Unnamed: 0  RID  PROGRESSES    5    6    7    8     9    10    11  ...  \
0           0  173         0.0  150  150  350  352  5258  6796  1169  ...   
1           1  307         0.0  149  154  347  354  5277  6769  1169  ...   
2           5  397         0.0  150  153  350  350  5269  6813  1174  ...   

     133    134   135   136   137   138   139   140   141   142  
0  11231  11021  5701  6353  8791  8895  3337  3388  1048  1008  
1  11278  11041  5774  6343  8861  8864  3331  3397  1052  1009  
2  11233  10983  5753  6359  8901  8895  3367  3420  1047  1009  

[3 rows x 133 columns]


In [102]:
adni_progs_pruned = adni_progs_pruned.drop(["Unnamed: 0","RID","PROGRESSES"], axis =1)
adni_stable_pruned = adni_stable_pruned.drop(["Unnamed: 0","RID","PROGRESSES"], axis =1)
print(adni_stable_pruned)

       5    6    7    8     9    10    11    12     13     14  ...    133  \
0    150  150  350  352  5258  6796  1169  1277  32094  32427  ...  11231   
1    149  154  347  354  5277  6769  1169  1285  32187  32517  ...  11278   
2    150  153  350  350  5269  6813  1174  1275  32176  32449  ...  11233   
3    150  153  347  355  5258  6757  1172  1280  32014  32353  ...  11192   
4    146  152  352  360  5281  6845  1174  1282  32313  32679  ...  11298   
5    150  153  350  356  5273  6781  1174  1283  32009  32318  ...  11194   
6    151  152  347  351  5286  6818  1174  1278  32295  32582  ...  11265   
7    151  154  347  356  5267  6813  1173  1283  32059  32475  ...  11227   
8    151  152  348  356  5292  6854  1171  1285  32541  32847  ...  11357   
9    150  154  350  354  5277  6801  1174  1284  32221  32675  ...  11292   
10   151  152  350  355  5289  6789  1172  1277  32160  32557  ...  11272   
11   152  154  352  355  5281  6813  1170  1280  32198  32524  ...  11240   

In [103]:
nacc_progs_pruned = pd.read_csv("/data2/MRI_PET_DATA/graph/NACC/Morph/mri_atlas/morph/roi/NACC_progressors_pruned_vol.csv")
nacc_stable_pruned = pd.read_csv("/data2/MRI_PET_DATA/graph/NACC/Morph/mri_atlas/morph/roi/NACC_stable_pruned_vol.csv")
# print(nacc_progs_pruned)


In [104]:
nacc_progs_pruned = nacc_progs_pruned.drop(["Unnamed: 0","RID","PROGRESSES"], axis =1)
nacc_stable_pruned = nacc_stable_pruned.drop(["Unnamed: 0","RID","PROGRESSES"], axis =1)
print(nacc_stable_pruned)

       5    6    7    8     9    10    11    12     13     14  ...    133  \
0    152  152  349  355  5260  6772  1175  1281  32077  32353  ...  11237   
1    150  156  351  360  5269  6808  1171  1279  32194  32544  ...  11236   
2    143  153  351  358  5256  6780  1176  1279  32353  32714  ...  11218   
3    148  151  351  354  5296  6809  1172  1280  32403  32755  ...  11282   
4    148  155  349  355  5287  6845  1170  1282  32244  32559  ...  11288   
5    150  154  348  353  5269  6767  1174  1280  32199  32515  ...  11261   
6    151  154  350  355  5278  6765  1170  1281  32065  32366  ...  11209   
7    151  153  348  355  5275  6802  1167  1283  32169  32444  ...  11253   
8    148  156  347  357  5287  6787  1167  1282  32324  32596  ...  11280   
9    151  153  349  355  5272  6797  1170  1283  32212  32548  ...  11257   
10   135  150  351  355  5275  6784  1180  1277  32100  32493  ...  11245   
11   151  154  349  355  5261  6806  1171  1282  32215  32496  ...  11219   

In [105]:
# Assuming you have a DataFrame named adni_progs_pruned
# Calculate the mean for each column and create a DataFrame with a single row
mean_values_df = pd.DataFrame(adni_progs_pruned.mean()).T
mean_values_df.index = ['prog_mean']
# Display the new DataFrame
# print(mean_values_df)


In [106]:
stable_mean_values_df = pd.DataFrame(adni_stable_pruned.mean()).T
stable_mean_values_df.index = ['stable_mean']
print(stable_mean_values_df)

In [107]:
#to concat must have list of dataframes 
dfs_to_concat = [mean_values_df,stable_mean_values_df]

means_df = pd.concat(dfs_to_concat)
print(means_df)

                      5           6           7           8            9  \
prog_mean    149.811688  152.227273  348.525974  354.311688  5277.188312   
stable_mean  149.802564  152.561538  348.838462  354.802564  5277.712821   

                      10           11           12            13  \
prog_mean    6799.201299  1171.857143  1280.000000  32238.266234   
stable_mean  6801.817949  1171.900000  1280.635897  32239.579487   

                       14  ...           133           134          135  \
prog_mean    32577.461039  ...  11258.149351  11032.974026  5756.545455   
stable_mean  32577.974359  ...  11263.084615  11036.741026  5766.158974   

                     136          137          138          139          140  \
prog_mean    6356.844156  8858.071429  8897.357143  3344.824675  3405.733766   
stable_mean  6364.241026  8863.397436  8906.166667  3345.582051  3409.230769   

                     141          142  
prog_mean    1048.168831  1009.311688  
stable_mean  1048.6

In [115]:
#data is nearly identical 
t_stat,p_val = stats.ttest_ind(means_df.loc['prog_mean','5'],means_df.loc['stable_mean','5'])
print(t_stat)
print(p_val)

nan
nan


/tmp/ipykernel_1081734/2588920122.py:1: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  t_stat,p_val = stats.ttest_ind(means_df.loc['prog_mean','5'],means_df.loc['stable_mean','5'])
/data2/MRI_PET_DATA/graph/graph_env/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1250: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/data2/MRI_PET_DATA/graph/graph_env/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1250: RuntimeWarning: invalid value encountered in double_scalars
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


In [117]:
#find regions with the largest difference between means 
#iterate through columns to find significant differences 
pd.set_option('display.max_rows', None)
cols = []
p_values = []

for col in means_df.columns:
    prog = means_df.loc['prog_mean',col]
    stab = means_df.loc['stable_mean',col]

    t_stat,p_val = stats.ttest_ind(prog,stab)
    cols.append(col)
    p_values.append(p_val)


# Create a DataFrame to display columns and their p-values
results_df = pd.DataFrame({'Column': cols, 'P-Value': p_values})
print(results_df)

    Column  P-Value
0        5      NaN
1        6      NaN
2        7      NaN
3        8      NaN
4        9      NaN
5       10      NaN
6       11      NaN
7       12      NaN
8       13      NaN
9       14      NaN
10      15      NaN
11      16      NaN
12      17      NaN
13      18      NaN
14      21      NaN
15      22      NaN
16      27      NaN
17      28      NaN
18      29      NaN
19      30      NaN
20      31      NaN
21      32      NaN
22      33      NaN
23      34      NaN
24      37      NaN
25      38      NaN
26      39      NaN
27      40      NaN
28      41      NaN
29      42      NaN
30      43      NaN
31      44      NaN
32      45      NaN
33      46      NaN
34      47      NaN
35      48      NaN
36      49      NaN
37      50      NaN
38      51      NaN
39      52      NaN
40      53      NaN
41      54      NaN
42      55      NaN
43      56      NaN
44      57      NaN
45      58      NaN
46      59      NaN
47      60      NaN
48      61      NaN


/tmp/ipykernel_1081734/4216931774.py:11: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  t_stat,p_val = stats.ttest_ind(prog,stab)
/data2/MRI_PET_DATA/graph/graph_env/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1250: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/data2/MRI_PET_DATA/graph/graph_env/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1250: RuntimeWarning: invalid value encountered in double_scalars
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


In [113]:
#read paper on strucutral covaruance
#could always make the case that for some reason this is happening because of the MCI swelling that happens early in mci 

#I need to normalize these values by how big the region is supposed to be because larger regions will have larger differences 

#this assumes that the stable is bigger than the prog (so positive values mean the stable patients have larger brain volumes)

# Calculate the difference between 'stable_mean' and 'prog_mean' for each column
difference_df = means_df.loc['stable_mean'] - means_df.loc['prog_mean']

# Create a new DataFrame with two columns: column name and difference
result_df = pd.DataFrame({'Column Name': difference_df.index, 'Difference': difference_df.values})

# Sort the DataFrame by the absolute difference in descending order
result_df['Absolute Difference'] = abs(result_df['Difference'])
result_df = result_df.sort_values(by='Absolute Difference', ascending=False)

# Reset the index for a cleaner display
result_df.reset_index(drop=True, inplace=True)

# Display the result DataFrame
print(result_df)


    Column Name  Difference  Absolute Difference
0            94   16.091742            16.091742
1            93   13.383417            13.383417
2            71   11.899900            11.899900
3            51   11.466500            11.466500
4            81   11.040460            11.040460
5           113   10.810390            10.810390
6            52   10.274858            10.274858
7            82    9.771262             9.771262
8           135    9.613520             9.613520
9            64    9.566234             9.566234
10          130    9.388112             9.388112
11          138    8.809524             8.809524
12          126    7.864369             7.864369
13          125    7.437729             7.437729
14          136    7.396870             7.396870
15          119    6.773427             6.773427
16          129    6.623477             6.623477
17          120    5.816250             5.816250
18           60    5.493606             5.493606
19          114    5

In [87]:
#find which rows have the biggest difference 
import pandas as pd

# Assuming you have a DataFrame named means_df with 'prog_mean' and 'stable_mean' rows
# Replace this with your actual data

# Calculate the absolute difference between 'prog_mean' and 'stable_mean'
means_df['abs_diff'] = abs(means_df.loc['prog_mean'] - means_df.loc['stable_mean'])

# Sort the DataFrame by the absolute difference in descending order
sorted_df = means_df.sort_values(by='abs_diff', ascending=False)

# Reset the index for a cleaner display
sorted_df.reset_index(inplace=True)

# Display the sorted DataFrame
print(sorted_df)


         index           5           6           7           8            9  \
0    prog_mean  149.811688  152.227273  348.525974  354.311688  5277.188312   
1  stable_mean  149.802564  152.561538  348.838462  354.802564  5277.712821   

            10           11           12            13  ...           134  \
0  6799.201299  1171.857143  1280.000000  32238.266234  ...  11032.974026   
1  6801.817949  1171.900000  1280.635897  32239.579487  ...  11036.741026   

           135          136          137          138          139  \
0  5756.545455  6356.844156  8858.071429  8897.357143  3344.824675   
1  5766.158974  6364.241026  8863.397436  8906.166667  3345.582051   

           140          141          142  abs_diff  
0  3405.733766  1048.168831  1009.311688       NaN  
1  3409.230769  1048.625641  1009.079487       NaN  

[2 rows x 132 columns]


In [81]:
#calc nacc means 
nacc_mean_values_df = pd.DataFrame(nacc_progs_pruned.mean()).T
nacc_mean_values_df.index = ['nacc_prog_mean']

stable_nacc_mean_values_df = pd.DataFrame(nacc_stable_pruned.mean()).T
stable_nacc_mean_values_df.index = ['stable_nacc_mean']

In [82]:
#to concat must have list of dataframes 
nacc_dfs_to_concat = [nacc_mean_values_df,stable_nacc_mean_values_df]

nacc_means_df = pd.concat(nacc_dfs_to_concat)
print(nacc_means_df)

                           5           6           7           8            9  \
nacc_prog_mean    149.876923  151.976923  348.738462  354.407692  5274.376923   
stable_nacc_mean  149.626984  152.457672  348.478836  354.634921  5278.224868   

                           10           11           12            13  \
nacc_prog_mean    6787.676923  1171.476923  1283.561538  32185.530769   
stable_nacc_mean  6794.100529  1171.235450  1283.092593  32227.309524   

                            14  ...           133           134          135  \
nacc_prog_mean    32517.715385  ...  11247.915385  11021.030769  5750.584615   
stable_nacc_mean  32561.417989  ...  11255.650794  11029.576720  5761.973545   

                          136          137          138          139  \
nacc_prog_mean    6348.038462  8851.800000  8892.169231  3337.800000   
stable_nacc_mean  6351.981481  8865.166667  8902.830688  3344.695767   

                          140          141          142  
nacc_prog_mean    33

In [ ]:
#compare the means 

In [ ]:
#see which regions have the biggest differences between means 